# The main goal of this project is:
1. How much the senators spent with benefits paid for the governament
2. Found the company that have more sells for the senators
3. How much employee each senators have
4. How much participation each senators have 
5. Create a measure to get how much senators spent and how much they was participate of the votation
6. How much cost the senators employee per year

# Star Schema
Fact table with the goals
Dims: Senators, Companys, EmployeesCabinet, Employees, Benefits

In [1]:
# Imports
import os
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, IntegerType

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

22/06/20 23:36:23 WARN Utils: Your hostname, MacBook-Pro-de-Paulo-Bernardo.local resolves to a loopback address: 127.0.0.1; using 100.90.148.60 instead (on interface en0)
22/06/20 23:36:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/06/20 23:36:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Functions

In [5]:
def skipRows(numberOfRowsToSkip, path):
    return spark.sparkContext\
            .textFile(path) \
            .zipWithIndex() \
            .filter(lambda row: row[1] >= numberOfRowsToSkip) \
            .map(lambda row: row[0])

# List all remuneration and write in silver layer

In [6]:
spark.read.csv("database/bronze/votation_by_senator/votation_per_senator.csv", header=True).show(truncate=False)

+---+----------------+------------+-----------------------+------------+----------+----------------+-------------+---------------------+
|_c0|codigoPalamentar|CodigoSessao|CodigoSessaoLegislativa|NumeroSessao|DataSessao|HoraInicioSessao|CodigoMateria|IdentificacaoProcesso|
+---+----------------+------------+-----------------------+------------+----------+----------------+-------------+---------------------+
|0  |4981            |248238      |865                    |106         |2021-09-02|16:00:00        |148281       |8073180              |
|1  |4981            |20633       |842                    |247         |2009-12-15|14:00:00        |88888        |3021274              |
|2  |4981            |20635       |842                    |248         |2009-12-16|14:00:00        |87198        |2049031              |
|3  |4981            |20635       |842                    |248         |2009-12-16|14:00:00        |82930        |644136               |
|4  |4981            |20635       |842   

In [42]:
years = ["2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

first_entrance = True

for year in years:
    path = f"database/bronze/civil_servants_remuneration/{year}"
    #rdd = skipRows(1, path)
    df = spark.read.csv(path, 
                   encoding="ISO-8859-1", header=True, comment= "Ú", sep=";", 
                   ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True
                  )
    df = df.withColumn("FILE_YEAR", f.lit(year))
    if first_entrance:
        first_entrance = False
        output_df = df
    else:
        output_df = output_df.union(df)

In [51]:
output_df = spark.read.csv("database/bronze/remuneration", 
                   encoding="ISO-8859-1", header=True, comment= "Ú", sep=";", 
                   ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True
                  )
output_df = output_df.withColumn("FILE_YEAR", f.lit(2022))

In [52]:
output_df_columns_renamed = output_df.select(f.col("VÍNCULO").alias("VINCULO"),
                "CATEGORIA", 
                 "CARGO", 
                 f.col("REFERÊNCIA CARGO").alias("REFERENCIA_CARGO"),
                f.col("SÍMBOLO FUNÇÃO").alias("SIMBOLO_FUNCAO"),
                f.col("ANO EXERCÍCIO").alias("ANO_EXERCICIO"),
                f.col("LOTAÇÃO EXERCÍCIO").alias("LOTACAO_EXERCICIO"),
                f.col("TIPO FOLHA").alias("TIPO_FOLHA"),
                f.col("REMUN_BASICA"),
                f.col("VANT_PESSOAIS"),
                f.col("FUNC_COMISSIONADA"),
                f.col("GRAT_NATALINA"),
                f.col("HORAS_EXTRAS"),
                f.col("OUTRAS_EVENTUAIS"),
                f.col("ABONO_PERMANENCIA"),
                f.col("REVERSAO_TETO_CONST"),
                f.col("IMPOSTO_RENDA"),
                f.col("PREVIDENCIA"),
                f.col("FALTAS"),
                f.col("REM_LIQUIDA"),
                f.col("DIARIAS"),
                f.col("AUXILIOS"),
                f.col("VANT_INDENIZATORIAS"),
                 f.col("FILE_YEAR"))

In [53]:
cols_to_double = ["REMUN_BASICA", 
    "VANT_PESSOAIS", 
    "FUNC_COMISSIONADA", 
    "GRAT_NATALINA", 
    "HORAS_EXTRAS", 
    "OUTRAS_EVENTUAIS", 
    "ABONO_PERMANENCIA", 
    "REVERSAO_TETO_CONST", 
    "IMPOSTO_RENDA", 
    "PREVIDENCIA", 
    "FALTAS", 
    "REM_LIQUIDA", 
    "DIARIAS", 
    "AUXILIOS", 
    "VANT_INDENIZATORIAS"]

for col in cols_to_double:
    output_df_columns_renamed = output_df_columns_renamed.withColumn(col, f.regexp_replace(col, ",", ".").cast("double"))

In [54]:
output_df_columns_renamed.repartition(1).write.mode('overwrite').parquet("database/silver/remuneration/")

In [46]:
output_df_columns_renamed.printSchema()

root
 |-- VINCULO: string (nullable = true)
 |-- CATEGORIA: string (nullable = true)
 |-- CARGO: string (nullable = true)
 |-- REFERENCIA_CARGO: string (nullable = true)
 |-- SIMBOLO_FUNCAO: string (nullable = true)
 |-- ANO_EXERCICIO: string (nullable = true)
 |-- LOTACAO_EXERCICIO: string (nullable = true)
 |-- TIPO_FOLHA: string (nullable = true)
 |-- REMUN_BASICA: double (nullable = true)
 |-- VANT_PESSOAIS: double (nullable = true)
 |-- FUNC_COMISSIONADA: double (nullable = true)
 |-- GRAT_NATALINA: double (nullable = true)
 |-- HORAS_EXTRAS: double (nullable = true)
 |-- OUTRAS_EVENTUAIS: double (nullable = true)
 |-- ABONO_PERMANENCIA: double (nullable = true)
 |-- REVERSAO_TETO_CONST: double (nullable = true)
 |-- IMPOSTO_RENDA: double (nullable = true)
 |-- PREVIDENCIA: double (nullable = true)
 |-- FALTAS: double (nullable = true)
 |-- REM_LIQUIDA: double (nullable = true)
 |-- DIARIAS: double (nullable = true)
 |-- AUXILIOS: double (nullable = true)
 |-- VANT_INDENIZATORIAS:

# Write benefits

In [2]:
import pandas as pd
import csv
import io

In [3]:
for year in list(range(2008, 2023)):
    with open(f"database/bronze/benefits/despesa_ceaps_{year}.csv") as file:
        contents = file.readlines()[2:]
        pd_series = pd.Series(contents)
        df = pd_series.str.replace('"', "", regex=False).replace('\n', "", regex=True)
        #df_csv = pd.read_csv(df[0], sep=";")
        print(type(df[1]))
        #print(type(df_csv))
        df.to_csv(f"database/bronze/benefits_pandas/despesa_ceaps_{year}.csv")
        

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [14]:
for year in list(range(2008, 2023)):
    with open(f"database/bronze/benefits/despesa_ceaps_{year}.csv") as file:
        filedata = file.readlines()[1:]
        filedata = "".join(filedata).replace('"', '')
        with open(f"database/bronze/benefits_pandas/despesa_ceaps_{year}.csv", 'w', encoding="utf-8-sig") as file:
              file.write(filedata)
        
        #df_csv = pd.read_csv(f"database/bronze/benefits_pandas/despesa_ceaps_{year}.csv", sep=";", skiprows=2)
        #df_csv.to_csv(f"database/bronze/benefits_pandas/despesa_ceaps_{year}.csv", encoding="utf-8-sig", sep=";")
        

isFirstEntrance = True
for year in list(range(2008, 2023)):
    if isFirstEntrance:
        pdf = pd.read_csv(f"database/bronze/benefits/despesa_ceaps_{year}.csv", sep = ";", encoding="ISO-8859-1", skiprows = 1, quotechar="\"")
        isFirstEntrance = False
    else:
        df = pd.read_csv(f"database/bronze/benefits/despesa_ceaps_{year}.csv", sep = ";", encoding="ISO-8859-1", skiprows=1, quotechar= "\"")
        pdf = pd.concat([pdf, df])

In [15]:

schema = StructType([ 
    StructField("ANO", StringType(),True), \
    StructField("MES",StringType(),True), \
    StructField("SENADOR",StringType(),True), \
    StructField("TIPO_DESPESA",StringType(),True), \
    StructField("CNPJ_CPF", StringType(), True), \
    StructField("FORNECEDOR", StringType(), True), \
    StructField("DOCUMENTO", StringType(), True), \
    StructField("DATA", StringType(), True), \
    StructField("DETALHAMENTO", StringType(), True), \
    StructField("VALOR_REEMBOLSADO", StringType(), True), \
    StructField("COD_DOCUMENTO", StringType(), True)
  ])

benefits_raw_df = spark.read.csv(f"database/bronze/benefits_pandas/", schema, sep = ";", header=True)

In [16]:
benefits_silver_df = benefits_raw_df.withColumn("CNPJ_CPF", f.when(f.isnan(f.col("CNPJ_CPF")), "NONEXISTENT").otherwise(f.col("CNPJ_CPF")))
benefits_silver_df = benefits_raw_df.withColumn("DOCUMENTO", f.when(f.isnan(f.col("DOCUMENTO")), "NONEXISTENT").otherwise(f.col("DOCUMENTO")))
benefits_silver_df = benefits_raw_df.withColumn("DATA", f.when(f.isnan(f.col("DATA")), "NONEXISTENT").otherwise(f.col("DATA")))
benefits_silver_df = benefits_raw_df.withColumn("VALOR_REEMBOLSADO", f.regexp_replace(f.col("VALOR_REEMBOLSADO"), ",", ".").cast("Double"))

In [17]:
benefits_silver_df.where("COD_DOCUMENTO = 2179546").show(truncate=True, n = 5)

+----+---+-------+--------------------+------------------+--------------------+--------------+----------+------------+-----------------+-------------+
| ANO|MES|SENADOR|        TIPO_DESPESA|          CNPJ_CPF|          FORNECEDOR|     DOCUMENTO|      DATA|DETALHAMENTO|VALOR_REEMBOLSADO|COD_DOCUMENTO|
+----+---+-------+--------------------+------------------+--------------------+--------------+----------+------------+-----------------+-------------+
|2022|  5|ROMÁRIO|Aluguel de imóvei...|05.196.080/0001-16|Calleri Assessori...|109/00022473-0|20/04/2022|        null|          1407.56|      2179546|
+----+---+-------+--------------------+------------------+--------------------+--------------+----------+------------+-----------------+-------------+



In [25]:
benefits_silver_df.select("SENADOR").drop_duplicates().orderBy(f.col("SENADOR")).show(truncate=False, n = 2)

+------------------------------------------------------------------------------------------------------+
|SENADOR                                                                                               |
+------------------------------------------------------------------------------------------------------+
|null                                                                                                  |
| ALESSANDRA NOGUEIRA DA SILVA COSTA (Matrícula 324283, COMISSIONADO), Voo: 3722 - CGH/BSB - 22/05/2018|
+------------------------------------------------------------------------------------------------------+
only showing top 2 rows



In [84]:
benefits_silver_df.repartition(1).write.mode('overwrite').parquet("database/silver/benefits/")

# Write senators

In [29]:
df = spark.read.csv("database/bronze/senators", encoding="utf-8", header=True, sep=";", 
                   ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

In [30]:
senators = df.select(f.col("index").alias("row_number"), "codigoPalamentar", "NomeParlamentar", "NomeCompletoParlamentar")

In [32]:
senators.repartition(1).write.mode("overwrite").parquet("database/silver/senators/")

# Write votation by senator

In [23]:
df = spark.read.csv("database/bronze/votation_by_senator", encoding="ISO-8859-1", header=True, sep=",", 
                   ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

In [24]:
votation = df.select(f.col("_c0").alias("row_number"), "codigoPalamentar", "CodigoSessao", "CodigoSessaoLegislativa", "NumeroSessao"
         , "DataSessao", "HoraInicioSessao", "CodigoMateria", "IdentificacaoProcesso")

In [28]:
votation.select("DataSessao").drop_duplicates().show(n=5)

+----------+
|DataSessao|
+----------+
|2016-03-01|
|2018-05-28|
|2013-03-26|
|2015-05-19|
|2019-06-04|
+----------+
only showing top 5 rows



In [31]:
votation = votation.withColumn("DataSessao", f.to_date(f.col("DataSessao"), "yyyy-MM-dd"))
votation = votation.withColumn("Year", f.year(f.col("DataSessao")))

In [32]:
votation.printSchema()

root
 |-- row_number: string (nullable = true)
 |-- codigoPalamentar: string (nullable = true)
 |-- CodigoSessao: string (nullable = true)
 |-- CodigoSessaoLegislativa: string (nullable = true)
 |-- NumeroSessao: string (nullable = true)
 |-- DataSessao: date (nullable = true)
 |-- HoraInicioSessao: string (nullable = true)
 |-- CodigoMateria: string (nullable = true)
 |-- IdentificacaoProcesso: string (nullable = true)
 |-- Year: integer (nullable = true)



In [33]:
votation.repartition(1).write.mode("overwrite").parquet("database/silver/votation_by_senator/")